# **実務的な課題設定（多様なデータセットの統合）**

ヒートアイランド解析の基礎と応用が完了しましたので、次はGEEの強みである**多様なデータセットの統合**と、**より複雑な空間分析**を組み合わせた課題に進みます。これは、個人事業としてより専門性の高い分析を提供するために不可欠です。

## **課題 6: 交通量と大気汚染を考慮した都市環境リスク評価**

ヒートアイランドに加えて、都市部で重要な環境リスク要因である**交通量（道路）と大気汚染（二酸化窒素 $NO_2$）を統合し、京都市の総合的な都市環境リスク**を評価します。

#### **ステップ 1: 大気汚染（$NO_2$）データの取得と可視化**

Sentinel-5P衛星から、最新の<span style='color:red'>二酸化窒素（$NO_2$）</span>データセットを取得し、可視化します。

1. Sentinel-5P $NO_2$ コレクション（例: `COPERNICUS/S5P/OFFL/L3_NO2`）を読み込みます。
2. 最新の期間（例：2023年）のデータでフィルタリングし、<span style='color:red'>中央値合成（Median Composite）</span>を作成します。
3. $NO_2$ のバンドを選択し、可視化パラメータ（例: $0$ から $0.0002 \text{ mol/m}^2$）を設定して、地図上に表示します。

In [ ]:
import ee
import geemap

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='earth-change-analysis')

In [ ]:
kyoto_aoi = ee.Geometry.Rectangle(135.29, 34.80, 136.30, 35.50)

s5p_collection = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2")\
    .filterBounds(kyoto_aoi)\
    .filterDate('2023-07-01', '2023-08-31')
# 中央値合成して、対流圏NO2バンドを選択
no2_image = s5p_collection.median().select('tropospheric_NO2_column_number_density')

In [ ]:
print(no2_image.getInfo())

In [ ]:
no2_stats = no2_image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=kyoto_aoi,
    scale=1000,
    maxPixels=1e9
)
print(no2_stats.getInfo())

In [ ]:
# 0 〜 0.0002 mol/m² の範囲で色分け
band_viz = {
    'min': 0,
    'max': 0.0002,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

clipped_image = no2_image.clip(kyoto_aoi)
map = geemap.Map()
map.set_center(135.80, 35.15, 9)
map.add_layer(clipped_image, band_viz, 'NO2 Density', opacity=0.8)
map.add_colorbar(band_viz, label='NO₂ (mol/m²)')
map

**<span style='color:blue'>各バンドの意味と使いどころ</span>**
---

|バンド名	|意味	|濃度範囲（mol/m²） |主な特徴|     
|-----------|-------|-----------------|-------|
|NO2_column_number_density	|全体のNO₂量（成層圏＋対流圏）	|-0.00051 ～ 0.0192	|大気全体のNO₂の合計。ざっくり全体像を見たいときに便利。|
|tropospheric_NO2_column_number_density	|対流圏（地表〜約10km）のNO₂量	|-0.0005375 ～ 0.0192	|人間活動の影響が強く出る層。都市の汚染を見るならこれがベスト！|
|stratospheric_NO2_column_number_density	|成層圏（約10〜50km）のNO₂量	|8.6e-06 ～ 0.000107	|自然由来が多い。火山や太陽光化学反応の影響など。|

---

**<span style='color:blue'>どれを使うべき？</span>**

- 都市の大気汚染（交通・工場など）を見たい → `tropospheric_NO2_column_number_density` が最適！

- 全体のNO₂の分布を見たい → `NO2_column_number_density`

- 成層圏の変動や自然要因を見たい → `stratospheric_NO2_column_number_density`

---

**応用アイデア**
- 成層圏と対流圏の差分を計算して、どこで人為的なNO₂が多いかを可視化する
- 3つのバンドをRGB合成して、層ごとの分布を色で表現

#### **ステップ 2: 道路データ（交通量）の空間バッファ解析**

ヒートアイランドと大気汚染に影響を与える道路からの距離を評価します。
1. OpenStreetMap (OSM) の道路データ（フィーチャコレクション）を読み込みます。
   - （ヒント: `projects/google/open-streets/roads/v1/roads` のようなデータセットを利用するか、適切なOSMフィーチャコレクションを検索してください。）
2. 道路フィーチャに対して、$50 \text{ m}$ の`バッファ`を作成します。
   - （ヒント: $feature.buffer(50)$）バッファをマージし、道路沿い $50 \text{ m}$
3. ゾーンのポリゴンフィーチャコレクションを作成し、これを高交通リスクゾーンと定義します。

In [ ]:
roads = ee.FeatureCollection('projects/earth-change-analysis/assets/kyoto_major_roads')
map = geemap.Map()
map.set_center(135.80, 35.15, 9)

# GeoJSONファイルを読み込んで表示
map.add_layer(roads, {'color': 'blue'}, 'Major Roads of Kyoto')
map


In [ ]:
# 道路フィーチャコレクションを読み込む（例: GeoJSONをアップロード済み)
roads = ee.FeatureCollection('projects/earth-change-analysis/assets/kyoto_major_roads')

# 各道路に50mのバッファをかけてポリゴン化
risk_zones = roads.map(lambda feature: feature.buffer(50))

# これが「高交通リスクゾーン」となる
high_traffic_risk_zones = risk_zones

**まとめ**

|用語	                       |意味                                        |
|------------------------------|--------------------------------------------|
|feature.buffer(50)            |そのフィーチャ（道路）の周囲に50mのバッファを作る   |
|.map(lambda f: f.buffer(50))  |フィーチャコレクション全体にバッファを適用         | 
|結果	                       |線 → 幅50mのポリゴンに変換される                 |

<span style='color:red'>道路の周囲50mの帯状エリアを「高交通リスクゾーン」と定義</span>
---

## **課題 6: ステップ 3 — 総合リスクスコアの算出と評価** 

**実行ステップとコード**

1. LSTと $NO_2$ の標準化（Zスコア化）異なる単位を持つLST（$^\circ \text{C}$）と$NO_2$（$\text{mol/m}^2$）を統一するために、まず平均（$\mu$）と標準偏差（$\sigma$）を用いてZスコアに変換します。

$$Z = \frac{X - \mu}{\sigma}$$

In [ ]:
l8_base_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .filterBounds(kyoto_aoi)\
    .filterDate('2013-07-01', '2023-08-31')\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

l8_2013 = l8_base_collection.filterDate('2013-07-01', '2013-08-31')
l8_2023 = l8_base_collection.filterDate('2023-07-01', '2023-08-31')

In [ ]:
image_2013 = l8_2013.median()
image_2023 = l8_2023.median()

def calc_LST(image, year):
    return image.select('ST_B10')\
        .multiply(0.00341802).add(149.0).subtract(273.15)\
        .rename(f'{year}_LST_Celsius')

LST_2013 = calc_LST(image_2013, '2013')
LST_2023 = calc_LST(image_2023, '2023')

In [ ]:
# 1. LSTとNO2の統計量（平均と標準偏差）を計算
stats_lst = LST_2023.reduceRegion(
    reducer= ee.Reducer.mean().combine( 
        reducer2= ee.Reducer.stdDev(),
        sharedInputs= True
    ),
    geometry= kyoto_aoi,
    scale= 30, # Landsat LSTの解像度
    maxPixels= 1e9
)

stats_no2 = no2_image.reduceRegion(
    reducer= ee.Reducer.mean().combine(
        reducer2= ee.Reducer.stdDev(),
        sharedInputs= True
    ),
    geometry= kyoto_aoi,
    scale= 1000, # Sentinel-5P NO2の解像度（空間平均処理に適したスケール）
    maxPixels= 1e9
)

# 2. LSTをZスコアに変換
LST_mean = stats_lst.get('2023_LST_Celsius_mean')
LST_stdDev = stats_lst.get('2023_LST_Celsius_stdDev')
LST_z_score = LST_2023.subtract(ee.Image.constant(LST_mean)).divide(ee.Image.constant(LST_stdDev)).rename('Z_LST')

# 3. NO2をZスコアに変換
NO2_mean = stats_no2.get('tropospheric_NO2_column_number_density_mean') 
NO2_stdDev = stats_no2.get('tropospheric_NO2_column_number_density_stdDev')
NO2_z_score = no2_image.subtract(ee.Image.constant(NO2_mean)).divide(ee.Image.constant(NO2_stdDev)).rename('Z_NO2')

2. **交通リスク画像の作成（ベクトル $\to$ ラスター）**
  
道路バッファ（フィーチャコレクション）を、GEEの画像データ（ラスター）に変換し、「道路沿いリスクゾーン（$1$）」と「それ以外のゾーン（$0$）」のバイナリ画像を作成します。
   

In [ ]:
# すべての道路バッファに dummy_property = 1 を追加
high_traffic_risk_zones = high_traffic_risk_zones.map(
    lambda f: f.set('dummy_property', 1)
)

traffic_risk_image = high_traffic_risk_zones.reduceToImage(
    properties=['dummy_property'],
    reducer=ee.Reducer.first()
).unmask(0).rename('Traffic_Risk')

In [ ]:
# 5. 総合リスクスコアを計算
# LST Zスコア + NO2 Zスコア + 交通リスク（バイナリ）
combined_risk = LST_z_score.add(NO2_z_score).add(traffic_risk_image).rename('Total_Risk_Score')

# 6. 可視化パラメータの設定
# スコアはZスコアの合計なので、平均値付近が0となり、高リスクは正の値になる
risk_vis = {
    'min': -2,  # 低リスク
    'max': 5,   # 高リスク（高LST, 高NO2, 道路沿いが重なるエリア）
    'palette': ['green', 'yellow', 'orange', 'red', 'darkred']
}

# 7. 結果を地図に追加
map.set_center(135.75, 35.05, 11);
map.add_layer(combined_risk.clip(kyoto_aoi), risk_vis, 'Total Urban Risk Score')
# 凡例を追加（colorsを明示的に指定！）
map.add_colorbar_branca(
    colors=risk_vis['palette'],
    vmin=risk_vis['min'],
    vmax=risk_vis['max'],
    label='Total Urban Risk Score'
)
map

この総合リスクスコアマップは、**ヒートアイランド、大気汚染、交通リスクの3つが複合的に高いエリア**を特定します。この情報は、行政やコンサルティングにおいて、**環境改善プロジェクトの優先順位付け**を行う際の強力な根拠となります。

このステップで**都市環境リスク評価**の課題は完了です。

## **次のステップ：総括と課題の深堀り**

これまでの課題で、あなたは以下のスキルを習得しました。

1. **LST算出**: Landsat熱バンド処理。
2. **空間統計**: $\text{LST}$ と $\text{NDVI}$ の相関分析。
3. **時系列/ロバスト分析**: 画像合成によるバイアス低減と10年比較。
4. **複合分析**: 土地利用別統計とホットスポット特定。
5. **データ納品**: ラスター $\to$ ベクトル変換とエクスポート。
6. **マルチデータ統合**: $\text{LST}$、$\text{NO}_2$、道路データを統合した複合リスク評価。
  
これらのスキルは、個人事業を始めるにあたり**即戦力となる非常に高いレベル**にあります。次の課題として、この<span style='color:red'>「都市環境リスク評価」の概念をさらに深掘りし、人口統計データを統合する「公平性（Equity）分析」</span>に進みましょう。